In [1]:
import torch
import torch.nn as nn
import torch.onnx
import onnx
import onnxruntime
import numpy as np

In [2]:
##AI
class NN(nn.Module) :
    def __init__(self, embedding_tensor) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=True, padding_idx=0)
        self.rnn = nn.LSTM(embedding_tensor.shape[1],10,batch_first = True)
        self.f = nn.Sequential(
            nn.Linear(10,2),
        )
    def forward(self, x) :
        x = self.embedding(x)
        x, h = self.rnn(x)
        x = x[:,-1,:]
        x = self.f(x)
        return x

In [3]:
torch_F = torch.load("korean_movie.pt", weights_only=False)
print(torch_F)

NN(
  (embedding): Embedding(30187, 200, padding_idx=0)
  (rnn): LSTM(200, 10, batch_first=True)
  (f): Sequential(
    (0): Linear(in_features=10, out_features=2, bias=True)
  )
)


In [5]:
x = torch.randint(0,30187,size = (1,30))

torch_F.eval()
torch.onnx.export(
    torch_F,
    x,
    "korean_movie_onnx.onnx",
    export_params = True,
    opset_version = 10,
    do_constant_folding = True,
    input_name = ["input"],
    output_name = ["output"],
    dynamic_axes= {"input" : {0 : "b"}, "output" : {0 : "b"}}
)

C:\ProgramData\anaconda3\Lib\site-packages\torch\onnx\utils.py:1824: UserWarning: Provided key input for dynamic axes is not a valid input/output name
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torch\onnx\utils.py:1824: UserWarning: Provided key output for dynamic axes is not a valid input/output name
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [6]:

onnx_F = onnx.load("korean_movie_onnx.onnx")
onnx.checker.check_model(onnx_F)

In [7]:
numpy_x = np.random.randint(0, 30187, size = (1,30)).astype(np.int64)
torch_x = torch.tensor(numpy_x, dtype = torch.long)

onnx_F = onnxruntime.InferenceSession("korean_movie_onnx.onnx", providers=["CPUExecutionProvider"])

onnx_x = {onnx_F.get_inputs()[0].name : numpy_x}
onnx_y = onnx_F.run(None, onnx_x)

torch_y = torch_F(torch_x)

print("torch 계산값 : ", torch_y.detach().numpy())
print("onnx 계산값 : ", onnx_y[0])

torch 계산값 :  [[ 0.86239684 -1.306062  ]]
onnx 계산값 :  [[ 0.8623967 -1.3060617]]
